<a href="https://colab.research.google.com/github/akm2208/Aayushi-First-Files/blob/main/Aayushi_Mishra_Midterm_Question_2_Temperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Temperature Projections from CMIP6 Models

Install Libraries

In [ ]:
!pip install cftime

In [2]:
!pip install  netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 14.1 MB/s eta 0:00:00


Import Libraries

In [3]:
import xarray as xr
import numpy as np
import pandas as pd
import netCDF4 as nc
import glob
import os
import cftime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Merge ..nc files

Open Dataset and Plot

In [ ]:
temp_Hist = "/content/drive/My Drive/CIVE5363_Midterm/tas_day_CanESM5_historical_r1i1p1f1_gn_18500101-20141231.nc"
temp_Hist= xr.open_dataset(temp_Hist, engine='netcdf4')


print(temp_Hist)
T = temp_Hist.sel(time = '1850-01-01')
T['tas'].plot()

In [8]:
temp_Hist =temp_Hist['tas'] - 273.15 #converting unit from Kelvin to Deg Celsius
temp_Hist['lon'] = xr.where(temp_Hist['lon'] > 180, temp_Hist['lon'] - 360, temp_Hist['lon'])
temp_Hist = temp_Hist.sortby('lon')

In [ ]:
temp_Hist

In [ ]:
T = temp_Hist.sel(time = '1980-04-07')
T.plot()


In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd

# Load the shapefile for continent borders
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Select the data
T = temp_Hist.sel(time='1980-04-07')

# Plot the data
plt.figure(figsize=(10, 6))
T.plot()

# Plot continent borders
world.boundary.plot(ax=plt.gca(), linewidth=1.5, color='black')

# Show the plot
plt.show()


Analyzing Global Mean, Max,Trends

In [ ]:
mean_temp = temp_Hist.mean(dim='lat')
mean_temp = mean_temp.mean(dim = 'lon')
print(mean_temp)


In [ ]:
annual_mean  = mean_temp.groupby('time.year').mean('time')
print(annual_mean)

In [ ]:
df = annual_mean.to_dataframe()
df.tas.plot()
plt.show()

In [ ]:
import numpy as np
coefficients, residuals, _, _, _ = np.polyfit(range(len(df.index)),df,1,full=True)

print('Slope ' + str(coefficients[0][1]))
print('Intercept ' + str(coefficients[1][1]))


x = range(1850, 2015)
y = ([coefficients[0]*x + coefficients[1] for x in range(len(df))])
plt.plot(df.tas)
plt.plot(x,y)
plt.ylim(2.5,7)

In [ ]:
# Question 2b(i)_Temperature
# I set the desiered latitude and longitudes
desired_lat = 35.8
desired_lon = -91

# To select the desired coordinates, I use .sel and the "nearest" method to ensure the closest values are selected
selected_location = temp_Hist.sel(lat=desired_lat, lon=desired_lon, method='nearest')

# I use groupby to calculate annual mean of the selected location
annual_mean = selected_location.groupby('time.year').mean('time')

# To plot the figure, I set the size to be 10,6 and set the title, axes names, and units
plt.figure(figsize=(10, 6))
annual_mean.plot()
plt.title('Annual Temperature from 1850-2014 (Latitude 35.8 and Longitude -91)')
plt.xlabel('Year')
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.show()